In [1]:
"""
Retrieve special sets of tweets given needs
"""

import sys, os

sys.path.append(os.path.abspath(os.path.join("..", "..", "src")))

%load_ext autoreload
%autoreload 2
import re
import time

import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

from common.database import Database
from common.app import App
from common.helpers import Helpers

app_run = App(debug=True)
db = Database("tweets.db", app=app_run)

In [2]:
# Get all tweets from db
with db:
    tws = db.get_all_tweets()
print(len(tws))
df_all = Helpers.df_from_db(tws)

238523


In [3]:
# OLD WAY, SHOULD NOT BE USED
# get only tweets about covid
df_yes = df_all[(df_all["covid_theme"] == 1) & ~(df_all["theme_hardcoded"] == "0")].copy()
print(f"{len(df_yes)=}")

len(df_yes)=88651


In [4]:
# New way, should be used
df_yes = Helpers.categorize_df_covid(df_all.copy())
print(f"{len(df_yes)=}")

len(df_yes)=101640


In [5]:
# select those that are NOT coded
df_uncoded = df_yes[
    ~(df_yes["topic"].isin(Helpers.topics_cov))
    & ~(df_yes["topic"].isin(Helpers.topics_not_cov))
].copy()
print(f"{len(df_uncoded)=}")

# and those that are coded
df_coded = df_yes[(df_yes["topic"].isin(Helpers.topics_cov))].copy()
print(f"{len(df_coded)=}")

len(df_uncoded)=16566
len(df_coded)=85074


In [6]:
# Convert date to be handled
df_uncoded["date"] = df_uncoded["created_at"].apply(Helpers.convert_date)
df_uncoded["date"] = pd.to_datetime(df_uncoded["date"], format="%d/%m/%Y")

df_coded["date"] = df_coded["created_at"].apply(Helpers.convert_date)
df_coded["date"] = pd.to_datetime(df_coded["date"], format="%d/%m/%Y")

df_all["date"] = df_all["created_at"].apply(Helpers.convert_date)
df_all["date"] = pd.to_datetime(df_all["date"], format="%d/%m/%Y")

df_yes["date"] = df_yes["created_at"].apply(Helpers.convert_date)
df_yes["date"] = pd.to_datetime(df_yes["date"], format="%d/%m/%Y")

In [7]:
# Only keep tweets in the time range
df_all_sorted = Helpers.sort_timerange(df_all.copy())
print(f"{len(df_all_sorted)=}") 

df_yes_sorted = Helpers.sort_timerange(df_yes.copy())
print(f"{len(df_yes_sorted)=}") 

df_uncoded_sorted = Helpers.sort_timerange(df_uncoded.copy())
print(f"{len(df_uncoded_sorted)=}") 

df_coded_sorted = Helpers.sort_timerange(df_coded.copy())
print(f"{len(df_coded_sorted)=}")

len(df_all_sorted)=185749
len(df_yes_sorted)=84233
len(df_uncoded_sorted)=2145
len(df_coded_sorted)=82088


## Special

## 1. Total count
In the period of interest, for each actor,  
      get total number of tweets  
      get total number of tweets about covid  

In [8]:
# Number of tweets per handle
g_all = pd.DataFrame(df_all_sorted.groupby("handle").size())
g_all = g_all.rename({0: "count_all"}, axis=1)

g_yes = pd.DataFrame(df_yes_sorted.groupby("handle").size())
g_yes = g_yes.rename({0: "count_covid"}, axis=1)

# Merge dataframes
count = g_all.join(g_yes, on="handle").fillna(0)
count["count_covid"] = count["count_covid"].astype(int) 

# Add columns
count["count_not_covid"] = count["count_all"] - count["count_covid"]
count["proportion"] = count["count_covid"] / count["count_all"]

In [9]:
count.sort_values("proportion", ascending=False)

,count_all,count_covid,count_not_covid,proportion
handle,,,,
@santeprevention,297,297,0,1.000000
@MinSoliSante,3060,3057,3,0.999020
@GUENGL,418,416,2,0.995215
@CVP_PDC,68,66,2,0.970588
@WHOSEARO,549,531,18,0.967213
...,...,...,...,...
@Left_EU,3532,336,3196,0.095130
@fedpolCH,341,27,314,0.079179
@Mitte_Centre,1134,80,1054,0.070547


In [10]:
# to excel 
count.to_excel("src/resources/data/2_count_actors.xlsx", index=False)

## 2. Weekly total count
In the period of interest, for each week, for each actor,  
      get total number of tweets  
      get total number of tweets about covid  

In [11]:
# Extract year and week
df_all_sorted["year"] = df_all_sorted["date"].dt.year
df_all_sorted["week"] = df_all_sorted["date"].dt.isocalendar().week
df_yes_sorted["year"] = df_yes_sorted["date"].dt.year
df_yes_sorted["week"] = df_yes_sorted["date"].dt.isocalendar().week

In [14]:
# Get tweets counts per handle per year per week
g_all = pd.DataFrame(df_all_sorted.groupby(["handle", "year", "week"]).size())
g_all = g_all.rename({0: "count_all"}, axis=1)

g_yes = pd.DataFrame(df_yes_sorted.groupby(["handle", "year", "week"]).size())
g_yes = g_yes.rename({0: "count_covid"}, axis=1)

# Merge dataframes
weekly_count = g_all.join(g_yes, on=["handle", "year", "week"]).fillna(0)

# Add columns
weekly_count["count_covid"] = weekly_count["count_covid"].astype(int)
weekly_count["count_not_covid"] = weekly_count["count_all"] - weekly_count["count_covid"]
weekly_count

count_all  count_covid  count_not_covid
handle           year week                                         
@10DowningStreet 2020 1             1            0                1
                      2             9            0                9
                      3            17            0               17
                      4            24            0               24
                      5            28            0               28
...                               ...          ...              ...
@vonderleyen     2021 10           16            6               10
                      11           30           12               18
                      12           35           17               18
                      13           18            6               12
                      53           11            6                5

[4371 rows x 3 columns]

In [13]:
# To excel
count.to_excel("src/resources/data/2_weekly_count_actors.xlsx", index=False)